<a href="https://colab.research.google.com/github/AhmedToto23/timetable-as-CSP/blob/main/part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#Import libraries
import pandas as pd
from collections import defaultdict
from openpyxl import Workbook
from openpyxl.utils import get_column_letter
from openpyxl.styles import Font, Alignment, PatternFill
import re

In [8]:
#Load Inputs
def load_inputs():
    timetable = pd.read_csv("/content/sample_data/final_timetable.csv")
    courses = pd.read_csv("/content/sample_data/Courses.csv")
     # Clean whitespace in object columns
    for col in courses.select_dtypes("object"):
        courses[col] = courses[col].str.strip()

    # Sections definition for CSIT department
    sections_map = {
        "CSIT": {
            "1": [{"name": f"s{i}", "count": 25} for i in range(1, 10)],
            "2": [{"name": f"s{i}", "count": 25} for i in range(1, 10)],
            "3": {
                "AID": [{"name": "s1", "count": 25}, {"name": "s2", "count": 25}, {"name": "s3", "count": 25}],
                "CNC": [{"name": "s1", "count": 25}, {"name": "s2", "count": 25}, {"name": "s3", "count": 25}],
                "CSC": [{"name": "s1", "count": 25}, {"name": "s2", "count": 25}],
                "BIF": [{"name": "s1", "count": 25}]
            },
            "4": {
                "AID": [{"name": "s1", "count": 25}, {"name": "s2", "count": 25}, {"name": "s3", "count": 25}],
                "CNC": [{"name": "s1", "count": 25}, {"name": "s2", "count": 25}, {"name": "s3", "count": 25}],
                "CSC": [{"name": "s1", "count": 25}, {"name": "s2", "count": 25}],
                "BIF": [{"name": "s1", "count": 25}]
            }
        }
    }
    return timetable, courses, sections_map

In [9]:
def get_colors():
    return [
        "FF6F61",  # Coral
        "6B5B95",  # Indigo
        "88B04B",  # Green
        "F7CAC9",  # Pink
        "92A8D1",  # Light Blue
        "955251",  # Mauve
        "B565A7",  # Orchid
        "009B77",  # Teal
        "DD4124",  # Red-Orange
        "45B8AC",  # Aqua
        "EFC050",  # Gold
        "5B5EA6",  # Royal Blue
        "9B2335",  # Crimson
        "DFCFBE",  # Beige
        "55B4B0",  # Mint
        "E15D44",  # Rust
        "7FCDCD",  # Light Teal
        "BC243C",  # Ruby
        "C3447A",  # Rose
        "98B4D4"   # Sky Blue
    ]

In [10]:
def get_time_structure():
    slots = ['9:00 AM', '10:45 AM', '12:30 PM', '2:15 PM']
    slot_ranges = ['9:00–10:30 AM', '10:45–12:15 PM', '12:30–2:00 PM', '2:15–3:45 PM']
    days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday']
    return days, slots, slot_ranges

In [11]:
#Excel Export Function
def export_excel(data, level, spec, section_def, all_sections, courses):
    level_num = level.split("-")[0] if "-" in level else level
    title = f"CSIT Level {level_num}" + (f"-{spec}" if spec else "")
    combined = spec is None and level_num in ["3", "4"]

    # Sections list
    if combined:
        sec_groups = all_sections["CSIT"][level_num]
        full_list = [f"CSIT-{level_num}-{k}-s{i+1}" for k, v in sec_groups.items() for i in range(len(v))]
        header_map = {k: len(v) for k, v in sec_groups.items()}
    else:
        full_list = [f"CSIT-{level_num}-{spec}-s{i+1}" if spec else f"CSIT-{level_num}-s{i+1}" for i in range(len(section_def))]
        header_map = {spec or 'Core': len(section_def)}

    # Filter rows
    pattern = "|".join(full_list)
    df = data[data["Sections"].str.contains(pattern, na=False)]

    course_names = dict(zip(courses["CourseID"], courses["CourseName"]))
    color_palette = get_colors()
    color_map = {cid: color_palette[i % len(color_palette)] for i, cid in enumerate(df["CourseID"].unique())}

    days, slots, ranges = get_time_structure()
    slot_index = {s: i for i, s in enumerate(slots)}

    df = df.sort_values(by=["Day", "StartTime"],
                        key=lambda c: c.map(lambda x: days.index(x) if c.name == "Day" else slot_index.get(x, 99)))

    # Build timetable dictionary
    table = defaultdict(lambda: defaultdict(tuple))
    merge_info = {}

    for _, row in df.iterrows():
        c_id, c_name = row["CourseID"], course_names.get(row["CourseID"], "")
        label = re.sub(r"([A-Z]+)(\d+)", r"\1 \2", c_id)
        text = f"{label} {c_name}\n{row['InstructorName']}\n{row['Type'][:3].upper()} {row['Room']}"
        color = color_map[c_id]
        relevant = [s for s in row["Sections"].split(", ") if s in full_list]

        if row["Type"].lower() == "lecture":
            merge_info[(row["Day"], row["StartTime"], text)] = ([r.split("-")[-1] for r in relevant], color)
        else:
            for sec in relevant:
                table[(row["Day"], row["StartTime"])][sec.split("-")[-1]] = (text, color)

    # Add merged
    for (day, time, text), (secs, col) in merge_info.items():
        for s in secs:
            table[(day, time)][s] = (text, col)

    # Build Excel
    wb = Workbook()
    ws = wb.active
    ws.title = title.replace(" ", "_")

    ws["A1"] = f"{title} Timetable"
    ws.merge_cells(start_row=1, start_column=1, end_row=1, end_column=len(full_list) + 2)
    ws["A1"].alignment = Alignment(horizontal="center")
    ws["A1"].font = Font(size=14, bold=True)

    ws["A2"], ws["B2"] = "Day", "Time"
    col = 3
    for header, count in header_map.items():
        ws.cell(row=2, column=col, value=f"CSIT {header}")
        ws.merge_cells(start_row=2, start_column=col, end_row=2, end_column=col + count - 1)
        for j in range(count):
            ws.cell(row=3, column=col + j, value=f"s{j+1}").font = Font(bold=True)
            ws.cell(row=3, column=col + j).alignment = Alignment(horizontal="center")
        col += count

    # Fill cells
    row = 4
    for day in days:
        start_row = row
        for idx, slot in enumerate(slots):
            ws.cell(row=row, column=1, value=day)
            ws.cell(row=row, column=2, value=ranges[idx])
            c = 3
            for sec in full_list:
                s_key = sec.split("-")[-1]
                content, bg = table[(day, slot)].get(s_key, ("", "FFFFFF"))
                cell = ws.cell(row=row, column=c, value=content)
                cell.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
                cell.fill = PatternFill(start_color=bg, end_color=bg, fill_type="solid")
                c += 1
            row += 1
        ws.merge_cells(start_row=start_row, start_column=1, end_row=row-1, end_column=1)
        ws.cell(row=start_row, column=1).alignment = Alignment(vertical="center", horizontal="center")

    # Formatting
    ws.column_dimensions["A"].width = 15
    ws.column_dimensions["B"].width = 16
    for i in range(3, len(full_list) + 3):
        ws.column_dimensions[get_column_letter(i)].width = 22

    filename = f"timetable_{level_num}{'_' + spec if spec else ''}.xlsx"
    wb.save(filename)
    print(f"✅ Excel timetable created: {filename}")

In [12]:
#Main Execution (Generate ALL)
def run_generator():
    try:
        timetable, courses, sections_map = load_inputs()
    except Exception as e:
        print("❌ Error loading files:", e)
        return

    print("Generating all timetables automatically...\n")

    for lvl, val in sections_map["CSIT"].items():
        if isinstance(val, list):
            print(f"🗂 Generating timetable for Level {lvl}...")
            export_excel(timetable, lvl, None, val, sections_map, courses)
        else:
            for spec, sec_def in val.items():
                print(f"🗂 Generating timetable for Level {lvl}-{spec}...")
                export_excel(timetable, lvl, spec, sec_def, sections_map, courses)

    print("\n✅ All Excel timetables generated successfully!")


# Run the generator
run_generator()

Generating all timetables automatically...

🗂 Generating timetable for Level 1...
✅ Excel timetable created: timetable_1.xlsx
🗂 Generating timetable for Level 2...
✅ Excel timetable created: timetable_2.xlsx
🗂 Generating timetable for Level 3-AID...
✅ Excel timetable created: timetable_3_AID.xlsx
🗂 Generating timetable for Level 3-CNC...
✅ Excel timetable created: timetable_3_CNC.xlsx
🗂 Generating timetable for Level 3-CSC...
✅ Excel timetable created: timetable_3_CSC.xlsx
🗂 Generating timetable for Level 3-BIF...
✅ Excel timetable created: timetable_3_BIF.xlsx
🗂 Generating timetable for Level 4-AID...
✅ Excel timetable created: timetable_4_AID.xlsx
🗂 Generating timetable for Level 4-CNC...
✅ Excel timetable created: timetable_4_CNC.xlsx
🗂 Generating timetable for Level 4-CSC...
✅ Excel timetable created: timetable_4_CSC.xlsx
🗂 Generating timetable for Level 4-BIF...
✅ Excel timetable created: timetable_4_BIF.xlsx

✅ All Excel timetables generated successfully!
